Установка необходимых библиотек

In [2]:
import sberpm

print("Версия SberPM ", sberpm.__version__)

Версия SberPM  3.4.0


In [1]:
import graphviz

Предобработка данных

In [3]:
import pandas as pd

df = pd.read_csv("HR_log_obezlich.csv", sep=";", encoding="utf-8")

print(df.columns)

Index(['case_id', 'line_id', 'from', 'to', 'time'], dtype='object')


In [4]:
df.head()

,case_id,line_id,from,to,time
0,1,1,Новый,Рассмотрение заказчиком,184.081136
1,1,2,Рассмотрение заказчиком,Оценка кандидата,216.771912
2,1,3,Оценка кандидата,Отклонен,222.700654
3,2,1,Новый,Рассмотрение заказчиком,528.787113
4,2,2,Рассмотрение заказчиком,Интервью,104.049962


In [5]:
from datetime import datetime, timedelta
import numpy as np

start_datetime = datetime(2024, 1, 1, 0, 0, 0)

df['start_time'] = None
df['end_time'] = None

for case in df['case_id'].unique():
    case_mask = df['case_id'] == case
    case_df = df[case_mask].sort_values(by='line_id')

    start_time = start_datetime
    start_times = []
    end_times = []

    for time in case_df['time']:
        start_times.append(start_time.strftime("%Y-%m-%d %H:%M:%S"))
        end_time = start_time + timedelta(hours=time)
        end_times.append(end_time.strftime("%Y-%m-%d %H:%M:%S"))
        start_time = end_time

    df.loc[case_mask, 'start_time'] = start_times
    df.loc[case_mask, 'end_time'] = end_times

case_offsets = {case: timedelta(hours=np.random.uniform(0.01, 0.1)) for case in df['case_id'].unique()}

df['start_time'] = pd.to_datetime(df['start_time']) + df['case_id'].map(case_offsets)
df['end_time'] = pd.to_datetime(df['end_time']) + df['case_id'].map(case_offsets)

df['start_time'] = df['start_time'].dt.strftime("%Y-%m-%d %H:%M:%S")
df['end_time'] = df['end_time'].dt.strftime("%Y-%m-%d %H:%M:%S")

In [53]:
df.head()

,case_id,line_id,from,to,time,start_time,end_time,duration_hours
0,1,1,Новый,Рассмотрение заказчиком,184.081131,2024-01-01 00:01:56,2024-01-08 16:06:48,662692
1,1,2,Рассмотрение заказчиком,Оценка кандидата,216.771912,2024-01-08 16:06:48,2024-01-17 16:53:06,780378
2,1,3,Оценка кандидата,Отклонен,222.700653,2024-01-17 16:53:06,2024-01-26 23:35:09,801723
3,2,1,Новый,Рассмотрение заказчиком,528.787109,2024-01-01 00:04:53,2024-01-23 00:52:06,1903633
4,2,2,Рассмотрение заказчиком,Интервью,104.049965,2024-01-23 00:52:06,2024-01-27 08:55:06,374580


Process Mining исследование с использованием библиотеки sberpm

In [6]:
from sberpm import DataHolder, DurationUnits, SuccessInputs

data_holder = DataHolder(
    data=df, 
    col_case="case_id", 
    col_stage="to",
    col_start_time="start_time",
    col_end_time="end_time",
    col_duration="time", 
    time_format="%Y-%m-%d %H:%M:%S", 
)

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



In [7]:
data_holder.data.head()

,case_id,line_id,from,to,time,start_time,end_time
0,1,1,Новый,Рассмотрение заказчиком,184.081131,2024-01-01 00:01:56,2024-01-08 16:06:48
1,1,2,Рассмотрение заказчиком,Оценка кандидата,216.771912,2024-01-08 16:06:48,2024-01-17 16:53:06
2,1,3,Оценка кандидата,Отклонен,222.700653,2024-01-17 16:53:06,2024-01-26 23:35:09
3,2,1,Новый,Рассмотрение заказчиком,528.787109,2024-01-01 00:04:53,2024-01-23 00:52:06
4,2,2,Рассмотрение заказчиком,Интервью,104.049965,2024-01-23 00:52:06,2024-01-27 08:55:06


In [8]:
from sberpm.miners import SimpleMiner
from sberpm.visual import GraphvizPainter

simple_miner = SimpleMiner(data_holder)
simple_miner.apply()
graph = simple_miner.graph

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

In [54]:
from sberpm.miners import AlphaMiner

alpha_miner = AlphaMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

In [11]:
from sberpm.miners import AlphaPlusMiner

alpha_plus_miner = AlphaPlusMiner(data_holder)
alpha_plus_miner.apply()
graph = alpha_plus_miner.graph

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

In [12]:
from sberpm.metrics import IdMetric

metrics = IdMetric(data_holder)
metrics.apply().head()

,trace,trace_length,unique_activities,unique_activities_num,loop_percent,total_duration,mean_duration,median_duration,max_duration,min_duration,probability,variance_duration,std_duration
1,"(Рассмотрение заказчиком, Оценка кандидата, От...",3,"{Рассмотрение заказчиком, Отклонен, Оценка кан...",3,0.000000,0.173209,0.057736,0.060214,0.061861,0.051134,0.001,0.000022,0.004717
2,"(Рассмотрение заказчиком, Интервью, Заполнение...",9,"{Рассмотрение заказчиком, Решение о найме, Инт...",9,0.000000,0.470658,0.052295,0.028903,0.146885,0.000241,0.001,0.002824,0.053138
3,"(Рассмотрение заказчиком, Отклонен)",2,"{Рассмотрение заказчиком, Отклонен}",2,0.000000,0.239366,0.119683,0.119683,0.180747,0.058619,0.001,0.003729,0.061064
4,"(Рассмотрение заказчиком, Планирование интервь...",11,"{Рассмотрение заказчиком, Решение о найме, Инт...",10,9.090909,0.402976,0.036634,0.014057,0.190110,0.000438,0.001,0.002737,0.052313
5,"(Рассмотрение рекрутером, Готов к переводу, Оф...",3,"{Рассмотрение рекрутером, Оформлен, Готов к пе...",3,0.000000,0.226572,0.075524,0.040362,0.185223,0.000987,0.001,0.006275,0.079217


In [13]:
from sberpm.metrics import TraceMetric

trace_metric = TraceMetric(data_holder, time_unit='d')
res = trace_metric.apply().sort_values('total_duration', ascending=False)
res.mean_duration.head()

⚠️ WARNING  | sberpm.metrics._trace_metric:	DataHolder instance has no user column. Skipping 'unique_users' metric calculation

⚠️ WARNING  | sberpm.metrics._trace_metric:	DataHolder instance has no user column. Skipping 'unique_users_num' metric calculation



(Рассмотрение заказчиком, Оценка кандидата, Отклонен)         0.013165
(Рассмотрение заказчиком, Планирование интервью, Отклонен)    0.013054
(Рассмотрение рекрутером, Резерв, Отклонен)                   0.052327
(Рассмотрение заказчиком, Отклонен)                           0.008773
(Рассмотрение рекрутером, Резерв, Самоотказ, Отклонен)        0.060300
Name: mean_duration, dtype: float64

In [14]:
from sberpm.metrics import ActivityMetric

activity_metric = ActivityMetric(data_holder)
nodes_count_metric = activity_metric.count().to_dict()
nodes_mean_metric = activity_metric.mean_duration().to_dict()

graph = simple_miner.graph
graph.add_node_metric('count', nodes_count_metric)
graph.add_node_metric('mean_duration', nodes_mean_metric)

painter = GraphvizPainter()
painter.apply(graph)
painter.show()

In [15]:
from sberpm.autoinsights import AutoInsights

auto_insights = AutoInsights(data_holder)

auto_insights.apply()

bool_insights = auto_insights.bool_insights()
print("Проблемные этапы (True - есть проблема):")
display(bool_insights)

float_insights = auto_insights.float_insights()
print("Оценка серьёзности проблем:")
display(float_insights)

summary = auto_insights.fin_effects_summary()
print("Выявленные проблемы в процессе найма:")
print(summary)

Проблемные этапы (True - есть проблема):


Метрика                       Длительность операции              \
Операция                         Растет со временем Bottle neck   
Рассмотрение заказчиком                       False       False   
Оценка кандидата                               True       False   
Отклонен                                      False       False   
Интервью                                       True       False   
Заполнение анкеты                              True       False   
Проверка                                       True       False   
Решение о найме                               False       False   
Согласование офера в компании                 False       False   
Готов к оформлению                             True       False   
Оформление                                     True       False   
Оформлен                                      False       False   
Планирование интервью                         False       False   
Рассмотрение рекрутером                        True       False   
Готов к переводу                              False       False   
Не выходит на связь                           False       False   
Резерв                                         True       False   
Самоотказ                                      True       False   

Метрика                                                                  \
Операция                      Нестандартизированная или ручная операция   
Рассмотрение заказчиком                                           False   
Оценка кандидата                                                  False   
Отклонен                                                          False   
Интервью                                                          False   
Заполнение анкеты                                                 False   
Проверка                                                          False   
Решение о найме                                                   False   
Согласование офера в компании                                     False   
Готов к оформлению                                                False   
Оформление                                                        False   
Оформлен                                                          False   
Планирование интервью                                             False   
Рассмотрение рекрутером                                           False   
Готов к переводу                                                  False   
Не выходит на связь                                               False   
Резерв                                                            False   
Самоотказ                                                         False   

Метрика                                                                 \
Операция                      Разовые инциденты Многократные инциденты   
Рассмотрение заказчиком                   False                  False   
Оценка кандидата                          False                  False   
Отклонен                                  False                  False   
Интервью                                  False                  False   
Заполнение анкеты                         False                  False   
Проверка                                  False                  False   
Решение о найме                           False                  False   
Согласование офера в компании             False                  False   
Готов к оформлению                        False                  False   
Оформление                                False                  False   
Оформлен                                  False                  False   
Планирование интервью                     False                  False   
Рассмотрение рекрутером                   False                  False   
Готов к переводу                          False                  False   
Не выходит на связь                       False                  False   
Резерв                                    

Оценка серьёзности проблем:


Метрика                       Длительность операции              \
Операция                         Растет со временем Bottle neck   
Рассмотрение заказчиком                    0.000000    0.459356   
Оценка кандидата                           0.001516    0.000000   
Отклонен                                   0.000935    1.000000   
Интервью                                   0.000948    0.434636   
Заполнение анкеты                          0.000948    0.421379   
Проверка                                   0.000959    0.000000   
Решение о найме                            0.000692    0.000000   
Согласование офера в компании              0.000946    0.496747   
Готов к оформлению                         0.001026    0.000000   
Оформление                                 0.000993    0.507137   
Оформлен                                   0.000947    0.768538   
Планирование интервью                      0.000947    0.586357   
Рассмотрение рекрутером                    1.000000    0.986984   
Готов к переводу                           0.000944    0.521946   
Не выходит на связь                        0.000948    0.565154   
Резерв                                     0.000950    0.564622   
Самоотказ                                  0.000949    0.624677   

Метрика                                                                  \
Операция                      Нестандартизированная или ручная операция   
Рассмотрение заказчиком                                        0.952314   
Оценка кандидата                                               0.000000   
Отклонен                                                       0.273456   
Интервью                                                       0.983355   
Заполнение анкеты                                              1.000000   
Проверка                                                       0.000000   
Решение о найме                                                0.000000   
Согласование офера в компании                                  0.905365   
Готов к оформлению                                             0.000000   
Оформление                                                     0.892319   
Оформлен                                                       0.564091   
Планирование интервью                                          0.792846   
Рассмотрение рекрутером                                        0.289800   
Готов к переводу                                               0.873723   
Не выходит на связь                                            0.819470   
Резерв                                                         0.820138   
Самоотказ                                                      0.744730   

Метрика                                                                 \
Операция                      Разовые инциденты Многократные инциденты   
Рассмотрение заказчиком                0.134950               0.459356   
Оценка кандидата                       0.068269               0.000000   
Отклонен                               0.061983               1.000000   
Интервью                               0.143937               0.434636   
Заполнение анкеты                      1.000000               0.421379   
Проверка                               0.367450               0.000000   
Решение о найме                        0.000000               0.000000   
Согласование офера в компании          0.053714               0.496747   
Готов к оформлению                     0.280407               0.000000   
Оформление                             0.066383               0.507137   
Оформлен                               0.146954               0.768538   
Планирование интервью                  0.298137               0.586357   
Рассмотрение рекрутером                0.409745               0.986984   
Готов к переводу                       0.062604               0.521946   
Не выходит на связь                    0.359719               0.565154   
Резерв                                 0.7

Выявленные проблемы в процессе найма:

                    Длительность следующих этапов увеличивается со временем, что может привести в дальнейшем к проблемам в процессе: «Оценка кандидата», «Интервью», «Заполнение анкеты», «Проверка», «Готов к оформлению», «Оформление», «Рассмотрение рекрутером», «Резерв», «Самоотказ».

                
                    Следующие этапы являются нерегулярными (редкими) и не требуются для успешной реализации процесса: «Готов к переводу», «Не выходит на связь», «Резерв». Максимальный потенциальный финансовый эффект при отказе от данных этапов 1617,66 рублей.

                
                    На следующих этапах наблюдается зацикленность, при которой экземпляр процесса начинается и заканчивается на один и тот же этап: «Рассмотрение заказчиком», «Оценка кандидата», «Отклонен», «Интервью», «Заполнение анкеты», «Проверка», «Решение о найме», «Согласование офера в компании», «Готов к оформлению», «Оформление», «Оформлен», «Планирование интервью», «Рас

In [16]:
from sberpm.metrics import ActivityMetric

activity_metric = ActivityMetric(data_holder)
activity_probs = activity_metric.apply()

activity_df = activity_probs.reset_index()[["index", "probability"]]
activity_df.columns = ["stage", "probability"]

In [17]:
from sklearn.svm import OneClassSVM

X = activity_df["probability"].values.reshape(-1, 1) 
svm = OneClassSVM(nu=0.1, kernel="rbf")
svm.fit(X)

activity_df["anomaly"] = svm.predict(X)
anomalous_stages = activity_df[activity_df["anomaly"] == -1]["stage"].tolist()

print("Аномальные этапы:", anomalous_stages)

successful_cases = data_holder.data.groupby("case_id")["to"].apply(set)

cases_without_anomalies = successful_cases.apply(lambda stages: all(stage not in stages for stage in anomalous_stages))

successful_without_anomalies = successful_cases[cases_without_anomalies]
print("Количество успешных экземпляров без аномальных этапов:", len(successful_without_anomalies))

Аномальные этапы: ['Интервью', 'Отклонен', 'Планирование интервью']
Количество успешных экземпляров без аномальных этапов: 31


In [18]:
from sberpm.imitation import Simulation

gen_holder = DataHolder(
    data=df, 
    col_case="case_id", 
    col_stage="to",
    col_start_time="start_time",
    col_end_time="end_time",
    col_duration="time",
    time_format="%Y-%m-%d %H:%M:%S"
)

simulation_original = Simulation(gen_holder)
simulation_original.generate()

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



  0%|          | 0/99 [00:00<?, ?it/s]

In [19]:
from sberpm.visual import Graph

def show_graph(graph: Graph, save=True, filename="graph", **kwargs):
    if kwargs.get("happy_path"):
        painter.apply_happy_path(graph, kwargs["happy_path"])
    elif kwargs.get("auto_insights"):
        painter.apply_insights(graph, **kwargs["auto_insights"])
    else:
        painter.apply(graph, **kwargs)

    display(painter.show())

In [20]:
from sberpm.metrics import TransitionMetric

as_is_miner = SimpleMiner(gen_holder)
as_is_miner.apply()

node_mean_duration = ActivityMetric(gen_holder, time_unit="second").mean_duration().round(2).to_dict()
edge_mean_duration = TransitionMetric(gen_holder, time_unit="second").mean_duration().round(2).to_dict()

as_is_graph = as_is_miner.graph
as_is_graph.add_node_metric("mean_time", node_mean_duration)

show_graph(as_is_graph, filename="Simulation_as-is", node_style_metric="mean_time", edge_style_metric="mean_time")

In [23]:
simulation_optimized = Simulation(gen_holder)

# 1. Снижение длительности проблемных этапов на 20%
problematic_stages = [
    "Оценка кандидата", "Интервью", "Заполнение анкеты", "Проверка", "Готов к оформлению", 
    "Оформление", "Рассмотрение рекрутером", "Резерв", "Самоотказ"
]
for stage in problematic_stages:
    simulation_optimized.scale_time_node(stage, scale=0.8)

# 2. Удаление нерегулярных этапов
excluded_stages = ["Готов к переводу", "Не выходит на связь", "Резерв"]
for stage in excluded_stages:
    simulation_optimized.delete_node(stage)

# 3. Устранение зацикливаний (ограничение повторов)
for stage in ["Интервью", "Заполнение анкеты"]:
    simulation_optimized.delete_loop(stage)

simulation_optimized.generate()

original_duration = simulation_original.get_result()["time"].mean()
optimized_duration = simulation_optimized.get_result()["time"].mean()

print(f"Исходная средняя длительность процесса: {original_duration:.2f} часов")
print(f"Оптимизированная средняя длительность процесса: {optimized_duration:.2f} часов")

optimized_miner = SimpleMiner(gen_holder)
optimized_miner.apply()
node_mean_duration_opt = ActivityMetric(gen_holder, time_unit="second").mean_duration().round(2).to_dict()
edge_mean_duration_opt = TransitionMetric(gen_holder, time_unit="second").mean_duration().round(2).to_dict()
optimized_graph = optimized_miner.graph
optimized_graph.add_node_metric("mean_time", node_mean_duration_opt)
show_graph(optimized_graph, filename="Simulation_optimized", node_style_metric="mean_time", edge_style_metric="mean_time")

  0%|          | 0/99 [00:00<?, ?it/s]

Исходная средняя длительность процесса: 346.94 часов
Оптимизированная средняя длительность процесса: 277.89 часов


RL методы

In [38]:
class ProcessEnvironment:
    def __init__(self, holder):
        self.holder = holder
        self.actions = holder.unique_stages.tolist()
        self.transition_probabilities = holder.stage.value_counts(normalize=True).to_dict()
        self.state = None

    def reset(self):
        self.state = random.choice(self.actions)
        return self.state

    def step(self, action):
        if action not in self.actions:
            return self.state, -10, True, {}  # Наказание за недопустимое действие

        reward = 0
        loss = self.holder.duration[self.holder.stage == action].mean()  # Потери = средняя длительность

        if action == "Оформлен":
            reward = 20  # Главная цель
        elif self.transition_probabilities.get(action, 0) < 0.05:
            reward = 10  # Редкие полезные переходы
        elif action == self.state:
            reward = -5  # Штраф за зацикливание
        else:
            reward = 1  # Обычный переход
        
        self.state = action  # Обновляем состояние
        return self.state, reward - loss / 10, action == "Оформлен", {}

In [40]:
class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.env = env
        self.q_table = {state: {action: 0 for action in env.actions} for state in env.actions}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.env.actions)
        else:
            return max(self.q_table[state], key=self.q_table[state].get, default=random.choice(self.env.actions))

    def train(self, episodes=1000):
        for _ in range(episodes):
            state = self.env.reset()
            done = False
            while not done:
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)
                best_next_action = max(self.q_table[next_state], key=self.q_table[next_state].get, default=0)
                self.q_table[state][action] += self.alpha * (reward + self.gamma * self.q_table[next_state][best_next_action] - self.q_table[state][action])
                state = next_state
        return self.q_table

In [41]:
class CrossEntropyMethod:
    def __init__(self, env, elite_frac=0.2, iterations=50):
        self.env = env
        self.elite_frac = elite_frac
        self.iterations = iterations

    def train(self):
        population = [random.sample(self.env.actions, len(self.env.actions)) for _ in range(100)]
        for _ in range(self.iterations):
            scores = [(self.fitness(seq), seq) for seq in population]
            elite_count = int(len(population) * self.elite_frac)
            elite = [seq for _, seq in sorted(scores, reverse=True)[:elite_count]]
            population = elite + [random.choice(elite) for _ in range(100 - elite_count)]
        return population[0]

    def fitness(self, sequence):
        return sum(self.env.step(action)[1] for action in sequence)

In [39]:
class GeneticAlgorithm:
    def __init__(self, env, population_size=20, mutation_rate=0.1, generations=50):
        self.env = env
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def initialize_population(self):
        return [random.sample(self.env.actions, len(self.env.actions)) for _ in range(self.population_size)]

    def fitness(self, sequence):
        return sum(self.env.step(action)[1] for action in sequence)

    def mutate(self, sequence):
        if random.uniform(0, 1) < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(sequence)), 2)
            sequence[idx1], sequence[idx2] = sequence[idx2], sequence[idx1]
        return sequence

    def train(self):
        population = self.initialize_population()
        for _ in range(self.generations):
            population = sorted(population, key=self.fitness, reverse=True)
            new_population = population[:self.population_size // 2]
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(population[:10], 2)
                crossover_point = random.randint(1, len(parent1) - 1)
                child = parent1[:crossover_point] + parent2[crossover_point:]
                new_population.append(self.mutate(child))
            population = new_population
        return population[0]

In [42]:
def train_pipeline(method, env, iterations=1000):
    if method == "qlearning":
        agent = QLearningAgent(env)
    elif method == "crossentropy":
        agent = CrossEntropyMethod(env, iterations=iterations)
    elif method == "genetic":
        agent = GeneticAlgorithm(env, generations=iterations//20)
    else:
        raise ValueError("Unsupported method")
    return agent.train()

In [45]:
holder = DataHolder(
    data=df,
    col_case="case_id",
    col_stage="to",
    col_start_time="start_time",
    col_end_time="end_time",
    col_duration="time",
    time_format="%Y-%m-%d %H:%M:%S"
)

# Сравнение методов
env = ProcessEnvironment(holder)
methods = ["qlearning", "crossentropy", "genetic"]
results = {}

for method in methods:
    print(f"Training with {method}...")
    results[method] = train_pipeline(method, env)

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена

Training with qlearning...
Training with crossentropy...
Training with genetic...


In [52]:
# Выбираем лучший метод
best_method = max(results, key=lambda k: sum(env.step(a)[1] for a in results[k]) / len(results[k]))
print(f"Best method: {best_method}")

# Запускаем симуляцию с лучшим методом
simulation = Simulation(holder)
for i in range(len(results[best_method]) - 1):
    state, next_state = results[best_method][i], results[best_method][i + 1]
    simulation.change_edge_probability(state, next_state, 1.0)

simulation.generate()

# Визуализация графа после обучения
generated_holder = DataHolder(
    data=simulation.get_result(),
    col_case="case_id",
    col_stage="to",
    col_start_time="start_time",
    col_end_time="end_time",
    col_duration="time",
    time_format="%Y-%m-%d %H:%M:%S"
)

optimized_miner = SimpleMiner(generated_holder)
optimized_miner.apply()
node_mean_duration_opt = ActivityMetric(generated_holder, time_unit="second").mean_duration().round(2).to_dict()
edge_mean_duration_opt = TransitionMetric(generated_holder, time_unit="second").mean_duration().round(2).to_dict()
optimized_graph = optimized_miner.graph
optimized_graph.add_node_metric("mean_time", node_mean_duration_opt)
optimized_graph.add_edge_metric("mean_time", edge_mean_duration_opt)

show_graph(optimized_graph, filename="Simulation_optimized", node_style_metric="mean_time", edge_style_metric="mean_time")

Best method: genetic


  0%|          | 0/99 [00:00<?, ?it/s]

ℹ️ INFO     | sberpm.baza._sota_utils:	Чтение данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных ивент лога...

ℹ️ INFO     | sberpm.baza._data_processing:	Определяем пропуски в текстовых данных...

ℹ️ INFO     | sberpm.baza._data_processing:	Данные будут отсортированы по следующим колонкам: ['case_id', 'start_time', 'end_time', 'to']

ℹ️ INFO     | sberpm.baza._data_processing:	Производится оптимизация типов данных...

ℹ️ INFO     | sberpm.baza.holder:	Обработка данных завершена



In [48]:
original_duration = simulation_original.get_result()["time"].mean()
optimized_duration = simulation.get_result()["time"].mean()

print(f"Исходная средняя длительность процесса: {original_duration:.2f} часов")
print(f"Оптимизированная средняя длительность процесса: {optimized_duration:.2f} часов")

Исходная средняя длительность процесса: 346.94 часов
Оптимизированная средняя длительность процесса: 188.41 часов
